In [1]:
import os
import pandas as pd


# Функция для обработки данных одной атаки
def process_attack_data(attack_type, csvs_folder):
    # Списки для нормального и атакующего трафика
    normal_files = os.listdir(os.path.join(csvs_folder, "normal"))
    attack_files = os.listdir(os.path.join(csvs_folder, "attack"))

    # Словарь для хранения корреляций по пакетным интервалам
    correlation_data = {}

    # Для каждого интервала пакетных выборок
    for normal_file in normal_files:
        # Определяем соответствующий файл для атаки
        packet_size = normal_file.split("_")[1]
        attack_file = [f for f in attack_files if f"_{packet_size}_" in f]

        if attack_file:
            # Чтение нормального и атакующего трафика
            normal_df = pd.read_csv(os.path.join(csvs_folder, "normal", normal_file))
            attack_df = pd.read_csv(os.path.join(csvs_folder, "attack", attack_file[0]))

            # Объединяем нормальные и атакующие данные
            combined_df = pd.concat([normal_df, attack_df])

            # Вычисление корреляционной матрицы
            corr_matrix = combined_df.corr()

            # Выделение корреляции с меткой "Label"
            target_corr = corr_matrix["Label"].abs().sort_values(ascending=False)

            # Сохраняем топ-5 коррелирующих признаков (исключая 'Label')
            top_features = target_corr.index[1:]  # 5 наиболее значимых признаков

            # Сохраняем корреляции для текущего интервала
            correlation_data[packet_size] = target_corr[top_features]

    # Преобразуем словарь в DataFrame, строки - это интервал пакетов, столбцы - признаки
    correlation_df = pd.DataFrame(
        correlation_data
    ).T  # Транспонируем для правильной формы

    # Преобразуем индекс из строк в числа для корректной сортировки
    correlation_df.index = correlation_df.index.astype(int)

    # Добавляем столбец с количеством признаков, имеющих корреляцию больше 0.7
    correlation_df['High Correlation Count'] = (correlation_df > 0.7).sum(axis=1)

    # Округление значений до 3 знаков после запятой
    correlation_df = correlation_df.round(3)

    # Сортировка по индексу (длине пакетной выборки)
    correlation_df = correlation_df.sort_index()

    # Вывод таблицы на экран
    print(f"Correlation table for attack type: {attack_type}")
    print(correlation_df.to_string(index=True))

    # Сохраняем DataFrame в CSV файл
    correlation_df.to_csv(
        f"{attack_type}_pearson_correlation_with_label.csv",
        index_label="Packet Interval",
    )

process_attack_data("inf-usb", "./csvs/")

Correlation table for attack type: inf-usb
      Average Packet Size  Flow Bytes/s  Flow IAT Mean  Fwd Header Length  Fwd IAT Min  Fwd IAT Std  Fwd Packet Length Max  Fwd Packet Length Mean  Max Packet Length  Total Length of Fwd Packets  High Correlation Count
10                  0.115         0.093          0.200              0.074        0.021        0.201                  0.075                   0.075              0.115                        0.115                       0
50                  0.166         0.098          0.417              0.099        0.042        0.419                  0.077                   0.079              0.166                        0.166                       0
100                 0.181         0.042          0.514              0.106        0.084        0.473                  0.032                   0.034              0.181                        0.181                       0
250                 0.189         0.072          0.579              0.117        

In [2]:
import os
import pandas as pd


# Функция для обработки данных одной атаки
def process_attack_data(attack_type, csvs_folder):
    # Списки для нормального и атакующего трафика
    normal_files = os.listdir(os.path.join(csvs_folder, "normal"))
    attack_files = os.listdir(os.path.join(csvs_folder, "attack"))

    # Словарь для хранения корреляций по пакетным интервалам
    correlation_data = {}

    # Для каждого интервала пакетных выборок
    for normal_file in normal_files:
        # Определяем соответствующий файл для атаки
        packet_size = normal_file.split("_")[1]
        attack_file = [f for f in attack_files if f"_{packet_size}_" in f]

        if attack_file:
            # Чтение нормального и атакующего трафика
            normal_df = pd.read_csv(os.path.join(csvs_folder, "normal", normal_file))
            attack_df = pd.read_csv(os.path.join(csvs_folder, "attack", attack_file[0]))

            # Объединяем нормальные и атакующие данные
            combined_df = pd.concat([normal_df, attack_df])

            # Вычисление корреляционной матрицы
            corr_matrix = combined_df.corr(method="spearman")

            # Выделение корреляции с меткой "Label"
            target_corr = corr_matrix["Label"].abs().sort_values(ascending=False)

            # Сохраняем топ-5 коррелирующих признаков (исключая 'Label')
            top_features = target_corr.index[1:]  # 5 наиболее значимых признаков

            # Сохраняем корреляции для текущего интервала
            correlation_data[packet_size] = target_corr[top_features]

    # Преобразуем словарь в DataFrame, строки - это интервал пакетов, столбцы - признаки
    correlation_df = pd.DataFrame(
        correlation_data
    ).T  # Транспонируем для правильной формы

    # Преобразуем индекс из строк в числа для корректной сортировки
    correlation_df.index = correlation_df.index.astype(int)

    # Добавляем столбец с количеством признаков, имеющих корреляцию больше 0.7
    correlation_df["High Correlation Count"] = (correlation_df > 0.7).sum(axis=1)

    # Округление значений до 3 знаков после запятой
    correlation_df = correlation_df.round(3)

    # Сортировка по индексу (длине пакетной выборки)
    correlation_df = correlation_df.sort_index()

    # Вывод таблицы на экран
    print(f"Correlation table for attack type: {attack_type}")
    print(correlation_df.to_string(index=True))

    # Сохраняем DataFrame в CSV файл
    correlation_df.to_csv(
        f"{attack_type}_spearman_correlation_with_label.csv",
        index_label="Packet Interval",
    )


process_attack_data("inf-usb", "./csvs/")

Correlation table for attack type: inf-usb
      Average Packet Size  Flow Bytes/s  Flow IAT Mean  Fwd Header Length  Fwd IAT Min  Fwd IAT Std  Fwd Packet Length Max  Fwd Packet Length Mean  Max Packet Length  Total Length of Fwd Packets  High Correlation Count
10                  0.098         0.005          0.003              0.050        0.061        0.004                  0.101                   0.101              0.098                        0.098                       0
50                  0.148         0.068          0.031              0.022        0.017        0.014                  0.077                   0.077              0.148                        0.148                       0
100                 0.165         0.081          0.043              0.022        0.063        0.057                  0.009                   0.008              0.165                        0.165                       0
250                 0.194         0.128          0.034              0.047        